In [4]:
import tensorflow as tf
import numpy as np
import base64
import keras
import os
import sys
from PIL import Image
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import csv

In [34]:
# Compress images

def resize(folder, fileName):
    filePath = os.path.join(folder, fileName)
    im = Image.open(filePath)
    w, h  = im.size
    newIm = im.resize((64, 64))
    newIm.save(filePath)

def bulkResize(imageFolder):
    imgExts = ["png", "jpg"]
    for path, dirs, files in os.walk(imageFolder):
        for fileName in files:
            ext = fileName[-3:].lower()
            if ext not in imgExts:
                continue
            resize(path, fileName)

imageFolder="/Users/Chrsitine/Desktop/Conjurers/images/" 
bulkResize(imageFolder)
print("Done resizing")

Done resizing


In [ ]:
# 0-indexing with one hot encoding is used for class labels:
idx_to_label = {0: "Coke zero", 1: "Oreo thins (original)", 2: "Cadbury chocolate fingers",
               3: "Diet coke", 4: "Hershey's drops", 5: "m&ms", 6: "Sun chips", 
               7: "Sour cream and onion Pringles", 8: "Coke (original)", 9: "Golden Oreo thins"}

In [30]:
# Load training and testing data 

x_train = []
y_train = []
x_test = []
y_test = []

imgExts = ["png", "jpg"]
counter = -1
for path, dirs, files in os.walk("/Users/Chrsitine/Desktop/Conjurers/images/"):
    for fileName in files:
        ext = fileName[-3:].lower()
        if ext not in imgExts:
            continue
        else:
            filePath = os.path.join(path, fileName)
            img = Image.open(filePath)
            img_array = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
            x_train.append(img_array)
            label = np.zeros(10)
            label[counter] = 1
            # y_train.append(label)
            y_train.append(counter)
    counter += 1

imgExts = ["png", "jpg"]
counter = -1
for path, dirs, files in os.walk("/Users/Chrsitine/Desktop/Conjurers/test_images/"):
    for fileName in files:
        ext = fileName[-3:].lower()
        if ext not in imgExts:
            continue
        else:
            filePath = os.path.join(path, fileName)
            img = Image.open(filePath)
            img_array = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
            x_test.append(img_array)
            label = np.zeros(10)
            label[counter] = 1
            # y_test.append(label)
            y_test.append(counter)
    counter += 1

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: " ,y_test.shape)

x_train.shape:  (550, 64, 64, 3)
y_train.shape:  (550,)
x_test.shape:  (10, 64, 64, 3)
y_test.shape:  (10,)


In [31]:
# Train model

batch_size = 56
num_classes = 10
epochs = 40

# input image dimensions
img_rows, img_cols = 64, 64

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Save trained model
model.save('trained_model.hdf5')
model.save_weights('trained_weights.hdf5')
print("Saved trained model")

score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

x_train shape: (550, 64, 64, 3)
y_train shape: (550,)
550 train samples
10 test samples
Train on 550 samples, validate on 10 samples
Epoch 1/40
550/550 [==============================] - 10s - loss: 2.5351 - acc: 0.1127 - val_loss: 2.2575 - val_acc: 0.1000
Epoch 2/40
550/550 [==============================] - 9s - loss: 2.2761 - acc: 0.1255 - val_loss: 2.2047 - val_acc: 0.3000
Epoch 3/40
550/550 [==============================] - 9s - loss: 2.1845 - acc: 0.2000 - val_loss: 2.1720 - val_acc: 0.2000
Epoch 4/40
550/550 [==============================] - 10s - loss: 2.1065 - acc: 0.2345 - val_loss: 2.0651 - val_acc: 0.2000
Epoch 5/40
550/550 [==============================] - 9s - loss: 2.0781 - acc: 0.2636 - val_loss: 1.8141 - val_acc: 0.6000
Epoch 6/40
550/550 [==============================] - 9s - loss: 1.8056 - acc: 0.3655 - val_loss: 1.5705 - val_acc: 0.3000
Epoch 7/40
550/550 [==============================] - 10s - loss: 1.8584 - acc: 0.3491 - val_loss: 1.7799 - val_acc: 0.3000
Epo

In [32]:
model.predict(x_test, verbose=0)

array([[  7.95562983e-01,   1.88833605e-02,   1.09651862e-02,
          5.21037094e-02,   1.05420448e-01,   1.52213452e-02,
          5.66586386e-05,   1.44844991e-03,   2.39636487e-04,
          9.81707199e-05],
       [  4.27160239e-06,   9.99953270e-01,   3.53988653e-05,
          6.88802629e-06,   2.18352767e-08,   2.89381887e-08,
          6.83768209e-10,   2.68876005e-10,   6.12759337e-08,
          9.50732115e-09],
       [  1.24132663e-01,   2.44672392e-02,   8.49384844e-01,
          1.87580287e-03,   2.39116360e-07,   1.25211627e-05,
          7.18544561e-06,   1.14374291e-06,   1.15605690e-04,
          2.78823291e-06],
       [  6.43543480e-03,   1.24299561e-03,   1.78609462e-05,
          2.84072328e-02,   1.87845799e-05,   8.74167963e-05,
          8.36183038e-03,   1.89294551e-05,   9.55389261e-01,
          2.03162053e-05],
       [  1.36764036e-04,   2.17551395e-04,   1.00692259e-05,
          2.19306414e-04,   9.99263585e-01,   4.91980209e-05,
          4.67419241e-08